# Text einlesen, encoden und als Enbeddings in Datenbank speichern

In [1]:
import fitz  # PyMuPDF
import tensorflow.compat.v1 as tf
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

In [2]:
# PDF mit fitz PyMuPDF einlesen
tf.disable_v2_behavior()

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    doc.close()
    return text

pdf_path = "C:/Users/rapha/Documents/Datascience/5. Semester/Artificial Intelligence/Einzelarbeit/Data/Der kleine Schmied.pdf"
extracted_text = extract_text_from_pdf(pdf_path)


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
# Modell für das Encoding bestimmen
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

# Text in kleinere Abschnitte aufteilen
text_segments = extracted_text.split("\n")  
embeddings = model.encode(text_segments)

In [4]:
# Embeddings in als Index in Datenbank schreiben
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2-Distanz
index.add(np.array(embeddings))
faiss.write_index(index, "faiss_index.db") 

# Mapping von Segment-IDs zum Text speichern
segment_mapping = {i: text for i, text in enumerate(text_segments)}

# Mapping als JSON speichern
with open("segment_mapping.json", "w", encoding="utf-8") as f:
    json.dump(segment_mapping, f)